In [12]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import chromedriver_autoinstaller

# Instala y ejecuta ChromeDriver compatible con tu versión de Chrome
chromedriver_autoinstaller.install()

# Configuración de Selenium
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

# Abre la página principal
driver.get("https://calidaddelaire.puebla.gob.mx/views/principal_monitoreo.php")
time.sleep(2)

# Click header "Indice de Calidad del aire"
driver.find_element(By.XPATH, "//a[@id='nv_ICA']").click()
time.sleep(2)

# Click drop menu "Historico Indice de Calidad del aire"
driver.find_element(By.XPATH, "//a[@data-bs-target='#modal_historialICA']").click()
time.sleep(2)

# Selecciona 'Por estación'
driver.find_element(By.XPATH, "//label[@for='his_filtro_ICA1']").click()
time.sleep(1)

# Selecciona la estación
estacion = Select(driver.find_element(By.XPATH, "//select[@id='his_estacion_ICA']"))
estacion.select_by_visible_text("AGUA SANTA")
time.sleep(1)

# Rango de fechas (de 2020-05-05 hasta hoy)
fecha_inicio = datetime(2020, 5, 5)
fecha_fin = datetime.now()

# DataFrame general para acumular todo
df_general = pd.DataFrame()

# Recorrer cada día
fecha_actual = fecha_inicio
while fecha_actual <= fecha_fin:
    try:
        # Formato requerido mm-dd-yyyy
        fecha_str = fecha_actual.strftime("%m-%d-%Y")

        # Selecciona la fecha
        fecha = driver.find_element(By.XPATH, "//input[@id='his_fecha_ICA']")
        fecha.clear()
        fecha.send_keys(fecha_str)
        time.sleep(1)

        # Envía la consulta
        driver.find_element(By.XPATH, "//button[@id='btnHistorial_ICA']").click()

        # Esperar a que cargue la tabla
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[@id='div_historial_ICA']//table"))
        )

        # Extrae las filas de la tabla
        filas = driver.find_elements(By.XPATH, "//div[@id='div_historial_ICA']//tbody/tr")

        datos = []
        for fila in filas:
            celdas = fila.find_elements(By.TAG_NAME, "td")
            datos.append([celda.text for celda in celdas])

        if datos:
            # Crear DataFrame del día
            columnas = ["Hora", "O3", "NO2", "CO", "SO2", "PM10", "PM2.5"]
            df_dia = pd.DataFrame(datos, columns=columnas)
            df_dia["Fecha"] = fecha_actual.strftime("%Y-%m-%d")

            # Agregar al DataFrame general
            df_general = pd.concat([df_general, df_dia], ignore_index=True)

            print(f"✅ Datos extraídos para {fecha_actual.strftime('%Y-%m-%d')}")
        else:
            print(f"⚠️ No hay datos para {fecha_actual.strftime('%Y-%m-%d')}")

    except Exception as e:
        print(f"❌ Error en la fecha {fecha_actual.strftime('%Y-%m-%d')}: {e}")

    # Avanza un día
    fecha_actual += timedelta(days=1)
    time.sleep(2)  # pequeña pausa para no saturar la página

# Reordenar columnas
df_general = df_general[["Fecha", "Hora", "O3", "NO2", "CO", "SO2", "PM10", "PM2.5"]]

# Guardar todos los datos en un solo archivo CSV
nombre_archivo = "aire_agua_santa_2020_2025_completo.csv"
df_general.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')
print(f"✅ Archivo guardado: {nombre_archivo}")

# Cerrar navegador
driver.quit()


❌ Error en la fecha 2020-05-05: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=136.0.7103.49); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7A048A145+76773]
	GetHandleVerifier [0x00007FF7A048A1A0+76864]
	(No symbol) [0x00007FF7A0248F7A]
	(No symbol) [0x00007FF7A0250978]
	(No symbol) [0x00007FF7A02539EC]
	(No symbol) [0x00007FF7A0253ABF]
	(No symbol) [0x00007FF7A0298F26]
	(No symbol) [0x00007FF7A02C735A]
	(No symbol) [0x00007FF7A0291CF6]
	(No symbol) [0x00007FF7A02C7570]
	(No symbol) [0x00007FF7A02EF0CF]
	(No symbol) [0x00007FF7A02C7133]
	(No symbol) [0x00007FF7A02904D1]
	(No symbol) [0x00007FF7A0291263]
	GetHandleVerifier [0x00007FF7A074A8ED+2962317]
	GetHandleVerifier [0x00007FF7A0744EC2+2939234]
	GetHandleVerifier [0x00007FF7A0762FF3+3062419]
	GetHandleVerifier [0x00007FF7A04A4B9A+1

KeyboardInterrupt: 